In [5]:
import timeatlas as ta
from timeatlas import TimeSeries, Metadata
from timeatlas import models
from pandas import Series, DatetimeIndex
import pandas as pd
import os
from pathlib import Path
import fbprophet

---
# Models

How to create models with TimeAtlas? At first let's create a TimeSeries.

In [7]:
index = DatetimeIndex(['2019-01-01', '2019-01-02', '2019-01-03', '2019-01-04'])
my_series = Series([0.4, 1.0, 0.7, 0.6], index=index)
ts = TimeSeries(my_series)
ts

## Grammar

Some definitions:

- `time series` : self-explanatory, i.e. the TimeSeries object
- `horizon` : the duration to predict after the last value of the time series
- `frequency`: the number of values per unit of time. Usually, the frequency is given in Pandas offset aliases (https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases)

```
                                horizon
   |-------------------------|- - - - - -|
                             |||||||||||||
           time series         frequency

```

In [ ]:
m = models.Prophet()

In [ ]:
m.fit(ts)

In [ ]:
Y_hat = m.predict('4 days')

In [ ]:
m.X_train.plot()
Y_hat.plot()

# NN-Models

In [1]:
from pandas import Series, DatetimeIndex
from torch.utils.data import DataLoader
from timeatlas import TimeSeriesDataset, TimeSeries
from timeatlas.models.NN import TimeSeriesClassificationDataset

### MLP Classification

In [2]:
index = DatetimeIndex(['2019-01-01', '2019-01-02', '2019-01-03', '2019-01-04'])
my_series = Series([0.4, 1.0, 0.7, 0.6], index=index)
ts = TimeSeries(my_series)
ts.label = 1
tsd = TimeSeriesDataset(data=[ts, ts])

In [3]:
dataset = TimeSeriesClassificationDataset(tsd)
dataloader = DataLoader(dataset, batch_size=1)

dataset.data

[tensor([0.4000, 1.0000, 0.7000, 0.6000], dtype=torch.float64),
 tensor([0.4000, 1.0000, 0.7000, 0.6000], dtype=torch.float64)]

In [4]:
for i in dataloader:
    print(i)

[tensor([[0.4000, 1.0000, 0.7000, 0.6000]], dtype=torch.float64), tensor([1])]
[tensor([[0.4000, 1.0000, 0.7000, 0.6000]], dtype=torch.float64), tensor([1])]


### LSTM Prediction

In [143]:
from pandas import Series, DatetimeIndex
from torch.utils.data import DataLoader
from timeatlas import TimeSeriesDataset, TimeSeries
from timeatlas.models import LSTMPrediction
from timeatlas.models.NN import TimeSeriesPredictionDataset

import torch.nn as nn
import torch

import numpy as np
import datetime

In [147]:
index = DatetimeIndex([datetime.datetime.fromtimestamp(t / 1e3) for t in np.arange(0, 100, 1)])
sine = np.sin(np.arange(0, 100, 1))
my_series = Series(sine, index=index)
ts = TimeSeries(my_series)
tsd = TimeSeriesDataset(data=[ts, ts])

In [166]:
dataset = TimeSeriesPredictionDataset(data=tsd, n=5)
dataloader = DataLoader(dataset, batch_size=20)


In [ ]:
model = LSTMPrediction(n_features=1, n_hidden=32, seq_len=5, n_layers=2).double()

In [ ]:
model

In [ ]:
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
epochs = 100

for i in range(epochs):
    for seq, labels in dataloader:
        
        model.reset_hidden_state()

        optimizer.zero_grad()

        y_pred = model(seq)
        
        single_loss = loss_function(y_pred, labels)
        single_loss.backward()
        optimizer.step()

    if i%25 == 1:
        print(f'epoch: {i:3} loss: {single_loss.item():10.8f}')

print(f'epoch: {i:3} loss: {single_loss.item():10.10f}')

In [ ]:
for v, k in dataloader:
    print(f'pred: {model(v)} truth {k}')

In [175]:
model

LSTMPrediction(
  (lstm): LSTM(1, 10, num_layers=2)
  (linear): Linear(in_features=10, out_features=1, bias=True)
)

In [176]:
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [177]:
epochs = 100

for i in range(epochs):
    for seq, labels in dataloader:
        
        model.reset_hidden_state()

        optimizer.zero_grad()

        y_pred = model(seq)
        
        single_loss = loss_function(y_pred, labels)
        single_loss.backward()
        optimizer.step()

    if i%25 == 1:
        print(f'epoch: {i:3} loss: {single_loss.item():10.8f}')

print(f'epoch: {i:3} loss: {single_loss.item():10.10f}')

epoch:   1 loss: 0.52704603
epoch:  26 loss: 0.52669155
epoch:  51 loss: 0.52637686
epoch:  76 loss: 0.52614753
epoch:  99 loss: 0.5260419940


In [178]:
seq

tensor([[-0.1761, -0.9235, -0.8218,  0.0354,  0.8601],
        [-0.9235, -0.8218,  0.0354,  0.8601,  0.8940],
        [-0.8218,  0.0354,  0.8601,  0.8940,  0.1060],
        [ 0.0354,  0.8601,  0.8940,  0.1060, -0.7795],
        [ 0.8601,  0.8940,  0.1060, -0.7795, -0.9483],
        [ 0.8940,  0.1060, -0.7795, -0.9483, -0.2453],
        [ 0.1060, -0.7795, -0.9483, -0.2453,  0.6833],
        [-0.7795, -0.9483, -0.2453,  0.6833,  0.9836],
        [-0.9483, -0.2453,  0.6833,  0.9836,  0.3796],
        [-0.2453,  0.6833,  0.9836,  0.3796, -0.5734]], dtype=torch.float64)

In [179]:
for v, k in dataloader:
    print(f'pred: {model(v)} truth {k}')

tensor([[ 0.0000,  0.8415,  0.9093,  0.1411, -0.7568],
        [ 0.8415,  0.9093,  0.1411, -0.7568, -0.9589],
        [ 0.9093,  0.1411, -0.7568, -0.9589, -0.2794],
        [ 0.1411, -0.7568, -0.9589, -0.2794,  0.6570],
        [-0.7568, -0.9589, -0.2794,  0.6570,  0.9894],
        [-0.9589, -0.2794,  0.6570,  0.9894,  0.4121],
        [-0.2794,  0.6570,  0.9894,  0.4121, -0.5440],
        [ 0.6570,  0.9894,  0.4121, -0.5440, -1.0000],
        [ 0.9894,  0.4121, -0.5440, -1.0000, -0.5366],
        [ 0.4121, -0.5440, -1.0000, -0.5366,  0.4202],
        [-0.5440, -1.0000, -0.5366,  0.4202,  0.9906],
        [-1.0000, -0.5366,  0.4202,  0.9906,  0.6503],
        [-0.5366,  0.4202,  0.9906,  0.6503, -0.2879],
        [ 0.4202,  0.9906,  0.6503, -0.2879, -0.9614],
        [ 0.9906,  0.6503, -0.2879, -0.9614, -0.7510],
        [ 0.6503, -0.2879, -0.9614, -0.7510,  0.1499],
        [-0.2879, -0.9614, -0.7510,  0.1499,  0.9129],
        [-0.9614, -0.7510,  0.1499,  0.9129,  0.8367],
        [-